# Notebook 07 – Memory + Router Agent

Objetivo:
Crear un sistema inteligente que:
- Recuerde contexto
- Decida qué pipeline usar
- Orqueste RAG, Graph RAG y Multiagentes


🟩 CELDA 2 — MEMORIA (TEORÍA)
Tipos de memoria

1️⃣ Short-term memory

Últimos mensajes

Conversación actual

2️⃣ Long-term memory

Vector DB

Usuarios

Historial

👉 Aquí empezamos solo con short-term (bien hecho)

📁backend/app/models/agents/memory.py

In [ ]:
class ConversationMemory:
    def __init__(self, max_messages: int = 5):
        self.max_messages = max_messages
        self.messages = []

    def add(self, role: str, content: str):
        self.messages.append({"role": role, "content": content})
        if len(self.messages) > self.max_messages:
            self.messages.pop(0)

    def get_context(self) -> str:
        return "\n".join(
            f"{m['role']}: {m['content']}" for m in self.messages
        )


🧠 Qué es esto de arriba

Memoria FIFO

No DB

Perfecta para empezar

100% controlable

📁backend/app/models/agents/router_agent.py

🔁 REEMPLAZA TODO

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from app.models.llms.groq_llm import get_groq_llm

llm = get_groq_llm()

router_prompt = ChatPromptTemplate.from_template("""
Eres un sistema experto que decide qué estrategia usar.

Opciones:
- rag
- graph_rag
- multiagent

Decide SOLO una opción.

Pregunta:
{question}
""")

def route(question: str) -> str:
    response = llm.invoke(router_prompt.format(question=question))
    decision = response.content.lower()

    if "graph" in decision:
        return "graph_rag"
    if "multi" in decision:
        return "multiagent"
    return "rag"

📁backend/app/services/content_service.py

🔁 REEMPLAZA EL CONTENIDO por esto (si ya tenía lógica simple)

In [ ]:
from app.models.agents.router_agent import route
from app.models.agents.memory import ConversationMemory
from app.services.rag_service import rag_answer, graph_rag_answer
from app.models.agents.marketing_agent import marketing_agent

memory = ConversationMemory()

def generate_content(question: str) -> str:
    memory.add("user", question)

    strategy = route(question)

    if strategy == "graph_rag":
        answer = graph_rag_answer(
            text=question,
            question=question
        )

    elif strategy == "multiagent":
        answer = marketing_agent(question)

    else:
        answer = rag_answer(question)

    memory.add("assistant", answer)
    return answer


🟩 CELDA  — CONTROLADOR (NO CAMBIA MUCHO

👉 Tu content_controller.py ya funciona
👉 Solo asegúrate de que llama a generate_content

Si ya lo hace → ✅ no toques nada

🧠 HE HECHO (MUY IMPORTANTE)

He creado:

✅ Memoria conversacional
✅ Router inteligente
✅ Orquestación de pipelines
✅ Sistema adaptable
✅ Base para producción

Esto es exactamente lo que usan:

copilots

chatbots avanzados

asistentes científicos